# Model Building for classification problems

***Akash Gupta***

## The process of computation

The **Carseats** dataset has been used for the purpose of demonstration here. The quantity we are trying to predict are sales of car seats (high or low), by using predictors such as - income, age, education level, urban/rural household and price of the component, among various other variables.

- First we compute the **information value** of all the predictors and make three different datasets based on different thresholds for the IV. 

- **Dummy variable encoding** is carried out for the categorical predictors and the numerical columns are **standardized** prior to subjecting the final dataframes to model fitting. 

- At first, the **logistic regressions** are fit on the three datasets with different number of features. We evaluate the model using 3 cases - AIC criterion, statistical t-test on the mean of cross validated accuracy scores and model variance as evidenced by the distribution of cross validated scores. 

- We also utilize the **grid search cv** functionality of Python to tune the hyperparameters of all the models before finalizing the model and fitting to the test dataset. 

- A **random forest** classifier is also fitted on the data for a more extensive comparison. 

## Results

- As per **AIC** values, it is found that the dataset with all the predictors is the best fit, as opposed to the datasets that had their features pruned out due to IV thresholds. 

- Even as per the **t-test** it seems that the statistical average accuracy of model3 is the best. However a caveat is that the Logit function in python is fit with a **regularization** hyperparameter that might be itself performing some level of variable selection. 

- The random forest classifier has markedly different results - even though the cross validated mean accuracy is higher for model3 - the statistical **t-test** of mean accuracy scores suggests that all three models are equal. So we might be tempted to choose the one with fewer features to avoid overfitting. 

In [661]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
sns.set_style('whitegrid')

In [514]:
df = pd.read_csv("/Users/Akashgupta/Desktop/data_files/cars.csv")
df['Sales'] = pd.qcut(df['Sales'], q=2, labels=[0, 1])

In [515]:
### VIEWING THE DATASET

df['Education'] = pd.qcut(df['Education'], q=3, labels = ['low', 'medium', 'high'])
df['Advertising'] = pd.qcut(df['Advertising'], q=2, labels = ['low', 'hi'])
df.pop('US')
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban
0,1,138,73,hi,276,120,Bad,42,high,Yes
1,1,111,48,hi,260,83,Good,65,low,Yes
2,1,113,35,hi,269,80,Medium,59,low,Yes
3,0,117,100,low,466,97,Medium,55,medium,Yes
4,0,141,64,low,340,128,Bad,38,medium,Yes


In [516]:
### A CUSTOM BUILT FUNCTION THAT COMPUTES INFORMATION VALUE OF ALL PREDICTORS

def iv(data, target, predictor):
    
    if predictor in list(set(list(data.columns)) - set(list(data.select_dtypes(include=['int', 'float']).columns))):
        d = pd.DataFrame([df[target], df[predictor]]).T
    elif predictor in list(df.select_dtypes(include=['int', 'float']).columns):
        bc = pd.qcut(np.log10(df[predictor]+3), q=6, labels=list(range(6)))
        d = pd.DataFrame([df[target], bc]).T
        
    bads = d[d[target]==0][target].count()
    goods = d[d[target]==1][target].count()
    
    g = pd.Series(d[d[target]==1][predictor].value_counts(), name='yes')
    g = g/goods
    
    b = pd.Series(d[d[target]==0][predictor].value_counts(), name='no')
    b = b/bads
    dd = pd.DataFrame([g, b]).T
    
    IV = (dd['yes'] - dd['no'])*np.log10(dd['yes']/dd['no'])
    return IV.sum()

In [517]:
### LOOP THAT PASSES PREDICTORS TO THE IV FUNCTION AND OUTPUTS THE VALUES

for pred in list(df.columns):
    if pred != 'Sales':
        if iv(df, 'Sales', pred) > 0.06:
            print(pred, ' --> ', iv(df, 'Sales', pred))

Advertising  -->  0.09772967097418558
Price  -->  0.35355502686045603
ShelveLoc  -->  0.3373246686942379
Age  -->  0.07621846277164837


In [519]:
### THREE DATASETS ARE INITIALIZED BASED ON DIFFERENT IV THRESHOLDS

data1 = df.loc[:, ['Price', 'ShelveLoc', 'Sales']]
data2 = df.loc[:, ['Price', 'ShelveLoc', 'Age', 'Advertising', 'Sales']]
data3 = df.loc[:, :]

In [229]:
data1.to_csv('/Users/Akashgupta/Desktop/data_files/data1.csv')
data2.to_csv('/Users/Akashgupta/Desktop/data_files/data2.csv')
data3.to_csv('/Users/Akashgupta/Desktop/data_files/data3.csv')

In [520]:
### THE SALES COLUMN IS INITIALIZED TO CATEGORICAL TYPE - RESPONSE COLUMN

data1['Sales'] = pd.Categorical(data1['Sales'])
data2['Sales'] = pd.Categorical(data2['Sales'])
data3['Sales'] = pd.Categorical(data3['Sales'])

In [533]:
### THIS FUNCTION IDENTIFIES ALL CATEGORICAL COLUMNS AND COMPUTES AND JOINS THEIR DUMMY ENCODED MATRICES
### ADDITIONALLY IT ALSO STANDARDIZES THE NUMERICAL VARIABLES AND COMPILES A FINAL DATASET

def final_frame(xx, target):
    
    X = xx.copy()
    cat_cols = list(set(list(X.columns)) - set(list(X.select_dtypes(include=['int', 'float']).columns)))
    q_cols = list(X.select_dtypes(include=['int', 'float']).columns)
    cat_cols.remove(target)
    initialized = cat_cols[0]
    cat_cols.remove(initialized)
    frame = pd.get_dummies(X[initialized], drop_first=True)
    
    Y = X[target]
    X.pop(target)

    for column in list(X.columns):
        if column in cat_cols:
            frame = frame.join(pd.get_dummies(X[column], drop_first=True))
        else:
            pass
        
    Qcols = []
    for col in q_cols:
        X[col] = (X[col] - X[col].mean())/X[col].std()
    
    frame2 = X[q_cols]
    finalframe = pd.concat([frame2, frame], axis=1)
    
    return finalframe, Y

In [534]:
### THE FUNCTION IS CALLED TO COMPUTE THE FINAL DATASETS AFTER FEATURE PROCESSING

x_data3, y3 = final_frame(data3, 'Sales')
x_data2, y2 = final_frame(data2, 'Sales')
x_data1, y1 = final_frame(data1, 'Sales')

In [539]:
### LOGISTIC REGRESSION MODELS ARE FIT OVER ALL THREE DATASETS

model1 = LogisticRegression(random_state=0, solver='liblinear')
res1 = model1.fit(x_data1, y1)

model2 = LogisticRegression(random_state=0, solver='liblinear')
res2 = model2.fit(x_data2, y2)

model3 = LogisticRegression(random_state=0, solver='liblinear')
res3 = model3.fit(x_data3, y3)

In [545]:
### THESE VARIABLES ENCODE THE PREDICTED VALUES BY THE 3 MODELS

yhat1 = model1.predict(x_data1)
yhat2 = model2.predict(x_data2)
yhat3 = model3.predict(x_data3)

In [574]:
### AKAIKE INFORMATION CRITERION IS CALCULATED FOR THE MODELS

aic1 = 2*log_loss(y1, model1.predict_proba(x_data1), normalize=False) + (2*len(list(x_data1.columns)))
aic2 = 2*log_loss(y2, model2.predict_proba(x_data2), normalize=False) + (2*len(list(x_data2.columns)))
aic3 = 2*log_loss(y3, model3.predict_proba(x_data3), normalize=False) + (2*len(list(x_data3.columns)))
print("AIC1: ", aic1)
print("AIC2: ", aic2)
print("AIC3: ", aic3)

AIC1:  395.3506626476684
AIC2:  339.8107946278777
AIC3:  247.12847794268163


In [595]:
### MODEL 1 - GRID SEARCH

param_grid = {'C': [1e-5, 1e-2, 1e0, 1e1, 1e10], 
              'penalty': ['l1', 'l2']}
grid1 = GridSearchCV(LogisticRegression(solver='liblinear'), cv=10, param_grid=param_grid)
grid1.fit(x_data1, y1)

GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [1e-05, 0.01, 1.0, 10.0, 10000000000.0],
                         'penalty': ['l1', 'l2']})

In [596]:
grid1.best_estimator_

LogisticRegression(penalty='l1', solver='liblinear')

In [597]:
grid1.best_score_

0.7375

In [598]:
### MODEL 2 -  GRID SEARCH

param_grid = {'C': [1e-5, 1e-2, 1e0, 1e1, 1e10], 
              'penalty': ['l1', 'l2']}
grid2 = GridSearchCV(LogisticRegression(solver='liblinear'), cv=10, param_grid=param_grid)
grid2.fit(x_data2, y2)

GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [1e-05, 0.01, 1.0, 10.0, 10000000000.0],
                         'penalty': ['l1', 'l2']})

In [599]:
grid2.best_estimator_

LogisticRegression(solver='liblinear')

In [600]:
grid2.best_score_

0.8

In [601]:
### MODEL3 - GRID SEARCH

param_grid = {'C': [1e-5, 1e-2, 1e0, 1e1, 1e10], 
              'penalty': ['l1', 'l2']}
grid3 = GridSearchCV(LogisticRegression(solver='liblinear'), cv=10, param_grid=param_grid)
grid3.fit(x_data3, y3)

GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [1e-05, 0.01, 1.0, 10.0, 10000000000.0],
                         'penalty': ['l1', 'l2']})

In [602]:
grid3.best_estimator_

LogisticRegression(C=10.0, penalty='l1', solver='liblinear')

In [603]:
grid3.best_score_

0.8774999999999998

In [634]:
### MODEL STANDARD DEVIATION

modelvar1 = cross_val_score(model1, x_data1, y1, cv=50).std()
modelvar2 = cross_val_score(model2, x_data2, y2, cv=50).std()
modelvar3 = cross_val_score(model3, x_data3, y3, cv=50).std()
print("variance1: ", modelvar1)
print("variance2: ", modelvar2)
print("variance3: ", modelvar3)

variance1:  0.16740669042783207
variance2:  0.132782717248895
variance3:  0.11726039399558574


In [630]:
scores1 = cross_val_score(model1, x_data1, y1, cv=50)
scores2 = cross_val_score(model2, x_data2, y2, cv=50)
scores3 = cross_val_score(model3, x_data3, y3, cv=50)

In [683]:
### T TEST INDICATING THAT MODEL3 IS THE BEST

stats.ttest_ind(scores3, scores1, equal_var=True, alternative='greater')

Ttest_indResult(statistic=4.623528928064958, pvalue=5.762119578316761e-06)

In [646]:
### FITTING THE FINAL MODEL ON DATA 3 - BEST

x3_train, x3_test, y3_train, y3_test = train_test_split(x_data3, y3, test_size=0.33, random_state=0)

In [678]:
logit3 = LogisticRegression(C = 10, penalty = 'l1', solver = 'liblinear')
fitted3 = logit3.fit(x3_train, y3_train)

In [679]:
logit3.score(x3_test, y3_test)

0.8939393939393939

In [651]:
### FITTING THE FINAL MODEL ON DATA2

x2_train, x2_test, y2_train, y2_test = train_test_split(x_data2, y2, test_size=0.33, random_state=0)

In [654]:
logit2 = LogisticRegression(solver = 'liblinear')
fitted2 = logit2.fit(x2_train, y2_train)

In [655]:
logit2.score(x2_test, y2_test)

0.8409090909090909

In [656]:
### FITTING THE FINAL MODEL ON DATA1

x1_train, x1_test, y1_train, y1_test = train_test_split(x_data1, y1, test_size=0.33, random_state=0)

In [659]:
logit1 = LogisticRegression(C = 1, penalty = 'l1', solver = 'liblinear')
fitted1 = logit1.fit(x1_train, y1_train)

In [660]:
logit1.score(x1_test, y1_test)

0.7575757575757576

In [662]:
######## RANDOM FOREST ###########

In [663]:
### MODEL 1 - GRID SEARCH

param_grid = {'n_estimators': [10, 20, 50, 100],
              'max_depth': [3, 4, 6, 7], 
              'min_samples_leaf': [30, 50, 70]}
rfgrid1 = GridSearchCV(RandomForestClassifier(), cv=10, param_grid=param_grid)
rfgrid1.fit(x_data1, y1)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 6, 7],
                         'min_samples_leaf': [30, 50, 70],
                         'n_estimators': [10, 20, 50, 100]})

In [664]:
rfgrid1.best_estimator_

RandomForestClassifier(max_depth=3, min_samples_leaf=30)

In [665]:
rfgrid1.best_score_

0.7449999999999999

In [667]:
### MODEL 2 - GRID SEARCH

param_grid = {'n_estimators': [10, 20, 50, 100],
              'max_depth': [3, 4, 6, 7], 
              'min_samples_leaf': [30, 50, 70]}
rfgrid2 = GridSearchCV(RandomForestClassifier(), cv=10, param_grid=param_grid)
rfgrid2.fit(x_data2, y2)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 6, 7],
                         'min_samples_leaf': [30, 50, 70],
                         'n_estimators': [10, 20, 50, 100]})

In [668]:
rfgrid2.best_estimator_

RandomForestClassifier(max_depth=6, min_samples_leaf=30, n_estimators=20)

In [669]:
rfgrid2.best_score_

0.78

In [670]:
### MODEL 3 - GRID SEARCH 

param_grid = {'n_estimators': [10, 20, 50, 100],
              'max_depth': [3, 4, 6, 7], 
              'min_samples_leaf': [30, 50, 70]}
rfgrid3 = GridSearchCV(RandomForestClassifier(), cv=10, param_grid=param_grid)
rfgrid3.fit(x_data3, y3)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 6, 7],
                         'min_samples_leaf': [30, 50, 70],
                         'n_estimators': [10, 20, 50, 100]})

In [671]:
rfgrid3.best_estimator_

RandomForestClassifier(max_depth=3, min_samples_leaf=30)

In [672]:
rfgrid3.best_score_

0.7975

In [675]:
rfscores1 = cross_val_score(RandomForestClassifier(max_depth=3, min_samples_leaf=30), x_data1, y1, cv=30)
rfscores2 = cross_val_score(RandomForestClassifier(max_depth=6, min_samples_leaf=30,
                                                   n_estimators=20), x_data2, y2, cv=30)
rfscores3 = cross_val_score(RandomForestClassifier(max_depth=3, min_samples_leaf=30), x_data3, y3, cv=30)

In [677]:
### MODEL STANDARD DEVIATION

print('RF1: ', rfscores1.std())
print('RF2: ', rfscores2.std())
print('RF3: ', rfscores3.std())

RF1:  0.1086438797180766
RF2:  0.11765086501818947
RF3:  0.09753568384493819


In [682]:
### T TEST RESULTING IN AN INCONCLUSIVE STATE - ALL MODELS ARE EQUALLY GOOD

stats.ttest_ind(rfscores3, rfscores1, alternative='greater')

Ttest_indResult(statistic=2.208991779259109, pvalue=0.01556963832053103)